In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

from sklearn.linear_model import LinearRegression
from sklearn.metrics        import r2_score, mean_squared_error
import numpy as np

In [ ]:
df = pd.read_csv("baseballcase_data.csv")

In [ ]:
# 2 REMOVE‑DUPLICATE COLUMNS & RENAME
# Some CSVs repeat column names (e.g., Year). Keep first occurrence only.
if df.columns.duplicated().any():
    df = df.loc[:, ~df.columns.duplicated()].copy()
    print("Duplicate column names removed →", len(df.columns), "unique columns remaining")

rename_map = {
    "SalaryADJ": "SalaryAdj",   # inflation‑adjusted free‑agent salary (millions)
    "Year Signed": "Year",      # contract‑signing year
}

for old, new in rename_map.items():
    if old in df.columns and new not in df.columns:
        df = df.rename(columns={old: new})

required = ["SalaryAdj", "Year", "HR"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Missing required column(s): {missing}. Check header names or update rename_map.")

In [ ]:
# 3 CONVERT TO NUMERIC & DROP MISSING VALUES
num_cols = required + [c for c in ["OBP", "SLG", "WAR"] if c in df.columns]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.dropna(subset=required)
print(f"Dataset after cleaning: {df.shape[0]} rows, {df.shape[1]} columns")

df["Year"] = df["Year"].astype(int)
#

Dataset after cleaning: 1878 rows, 274 columns


In [ ]:
# 4 NEW METRICS
df["log_salary"] = np.log(df["SalaryAdj"])

df["cost_per_HR"] = np.where(df["HR"] > 0, df["SalaryAdj"] / df["HR"], np.nan)

In [ ]:
# 5 SUMMARY TABLES
summary_cols = [c for c in ["SalaryAdj", "log_salary", "HR", "OBP", "SLG", "WAR"] if c in df.columns]
summary = df[summary_cols].describe()
summary.to_csv("overall_summary.csv")
print("overall_summary.csv saved")

corr_candidates = [c for c in ["SalaryAdj", "HR", "OBP", "SLG", "WAR"] if c in df.columns]
corr = df[corr_candidates].corr().loc[:, ["SalaryAdj"]].round(3)
corr.to_csv("salary_correlations.csv")
print("salary_correlations.csv saved")

overall_summary.csv saved
salary_correlations.csv saved


In [ ]:
# 6 VISUALISATIONS
plt.style.use("default")

# 6a Salary distribution (raw + log)
fig, ax = plt.subplots(figsize=(8, 6))
ax.hist(df["SalaryAdj"], bins=40, alpha=0.6, label="Salary ($M)")
ax.set_xlabel("Salary (inflation‑adjusted $M)")
ax.set_ylabel("Number of Players")
ax.set_title("Distribution of Free‑Agent Salaries (1998–2013)")

ax2 = ax.twinx()
ax2.hist(df["log_salary"], bins=40, color="orange", alpha=0.3, label="Log Salary")
fig.legend(loc="upper right")
fig.tight_layout()
fig.savefig("salary_distribution.png")
plt.close(fig)

# 6b Salary by year
plt.figure(figsize=(12, 6))
order = sorted(df["Year"].unique())
sns.boxplot(x="Year", y="SalaryAdj", data=df, order=order, showfliers=False)
plt.xticks(rotation=45)
plt.ylabel("Salary (inflation‑adjusted $M)")
plt.title("Free‑Agent Salaries by Signing Year")
plt.tight_layout()
plt.savefig("salary_by_year.png")
plt.close()

# 6c Correlation heat‑map
plt.figure(figsize=(6, 4))
plt.title("Correlation with SalaryAdj")
sns.heatmap(corr, annot=True, cmap="Blues", vmin=-1, vmax=1, cbar=False)
plt.tight_layout()
plt.savefig("corr_heatmap.png")
plt.close()

# 6d HR vs. Salary scatter
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(df["HR"], df["SalaryAdj"], alpha=0.6)
ax.set_xlabel("Home Runs (season before FA)")
ax.set_ylabel("Salary (inflation‑adjusted $M)")
ax.set_title("Salary vs. Home Runs for Free‑Agent Batters")

hr_cost = df.dropna(subset=["cost_per_HR"]).copy()
cheapest = hr_cost.nsmallest(1, "cost_per_HR").iloc[0]
priciest = hr_cost.nlargest(1, "cost_per_HR").iloc[0]
for row, color, label in [(cheapest, "green", "Cheapest per HR"), (priciest, "red", "Costliest per HR")]:
    ax.scatter(row["HR"], row["SalaryAdj"], color=color, s=110, edgecolor="black", zorder=5)
    ax.text(row["HR"], row["SalaryAdj"], row.get("Player", ""), fontsize=8, ha="left", va="bottom")

fig.tight_layout()
fig.savefig("hr_vs_salary.png")
plt.close(fig)

In [ ]:
# 7 HR‑COST TABLE
hr_cost_summary = hr_cost[["Player", "HR", "SalaryAdj", "cost_per_HR"]].sort_values("cost_per_HR")
hr_cost_summary.to_csv("hr_cost_table.csv", index=False)
print("hr_cost_table.csv saved")
print("Descriptive analysis complete – outputs generated.")

hr_cost_table.csv saved
Descriptive analysis complete – outputs generated.


In [ ]:
# --- 7. Predictive Modeling for 2013 Free Agents -------------------------

# 7a. Split: train on Year<2013, test on Year==2013
df_train = df[df["Year"] < 2013].copy()
df_2013  = df[df["Year"] == 2013].copy()

# 7b. Select predictors & target
predictors = ["HR", "OBP", "SLG", "WAR"]    # key performance metrics
X_train = df_train[predictors]
y_train = df_train["SalaryAdj"]
X_test  = df_2013[predictors]
y_test  = df_2013["SalaryAdj"]

# 7c. Fit Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

# 7d. Evaluate on 2013 hold-out
y_pred = lr.predict(X_test)
r2     = r2_score(y_test, y_pred)
n, p   = X_test.shape
adj_r2 = 1 - (1 - r2)*(n - 1)/(n - p - 1)
rmse   = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"2013 Test R²       : {r2:.3f}")
print(f"2013 Adj. R²       : {adj_r2:.3f}")
print(f"2013 Test RMSE (\$M): {rmse:.2f}")

# 7e. Feature importances (linear coefficients)
coef_df = (
    pd.Series(lr.coef_, index=predictors)
      .sort_values(key=lambda s: np.abs(s), ascending=False)
      .rename("Coefficient")
      .reset_index()
      .rename(columns={"index":"Feature"})
)
print("\nKey Drivers (by abs(coefficient)):\n", coef_df)

# 7f. Value potential in 2013
df_2013["PredictedSalary"] = y_pred
df_2013["ValuePotential"]  = df_2013["PredictedSalary"] - df_2013["SalaryAdj"]

# 7g. Top/Bottom 5 undervalued/overvalued
undervalued = (
    df_2013
      .nlargest(5, "ValuePotential")
      [ ["Player","Team","SalaryAdj","PredictedSalary","ValuePotential"] ]
)
overvalued = (
    df_2013
      .nsmallest(5, "ValuePotential")
      [ ["Player","Team","SalaryAdj","PredictedSalary","ValuePotential"] ]
)

undervalued.to_csv("undervalued_2013.csv", index=False)
overvalued .to_csv("overvalued_2013.csv",  index=False)

print("\nSaved top‐5 undervalued → undervalued_2013.csv")
print("Saved top‐5 overvalued  → overvalued_2013.csv")

2013 Test R²       : 0.082
2013 Adj. R²       : 0.037
2013 Test RMSE (\$M): 4524898.61

Key Drivers (by abs(coefficient)):
   Feature   Coefficient
0     OBP  2.095174e+07
1     SLG -1.016767e+07
2      HR  2.226217e+05
3     WAR -1.058950e+05

Saved top‐5 undervalued → undervalued_2013.csv
Saved top‐5 overvalued  → overvalued_2013.csv
